In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/구석구석_서울크롤링_4057.csv')

In [2]:
# 사용할 컬럼 선별
use_col = ['장소명', '상세설명','한줄설명', '리뷰','대표메뉴','취급메뉴', '주소']

# 전체 컬럼 중 사용할 컬럼이 아니면 제거
df.drop(set(df.columns) - set(use_col), axis=1, inplace=True)

In [3]:
# 주소 결측치 제거
df.drop(index=2665, inplace=True)

# index 재설정
df.reset_index(drop='index', inplace=True)

# null 값 공백으로 치환
df.fillna('', inplace=True)

In [4]:
from konlpy.tag import Komoran
import re
ko = Komoran()

for col in use_col[1:]:
    for row, text in enumerate(df[col]):
        # 이모지, 특수문자 제거
        token = ko.nouns(re.sub('[^A-Za-z0-9가-힣]', ' ', text))

        # 정제된 토큰들로 다시 문장화
        df.loc[row, col] = " ".join(token)

In [5]:
# 사용할 컬럼들의 모든 text를 이어 붙일 '정보' 컬럼 생성
df['정보'] = df['장소명'] + " "

for i in use_col[1:] :
    df.정보 += df[i]

# 장소명과 정보 컬럼을 제외한 나머지 컬럼 삭제
df.drop(use_col[1:], axis=1, inplace=True)

### 불용어 제거

In [6]:
# 불용어
stop_word = [
    ' ',',','.','등','수','이요','요','번','앞','곳','집','류','모','봉','돌','호선','객','부','콩','겸','그','위','계','무','안','눈','방','이','팅',
    '비','통','석','개','세','순','외','점','전','것','두','나','옥','내','역','과','뿐','이류','찰','더','여','년','로','층','차','종','배','중','몽',
    '도','날','널','때','꼭','및','볼','후','룸','의','실','또','제','온','를','터','인','탕','총','각','명','저','리','처','존','뜻','이자','움','입',
    '게','좀','데','기','만','용','초','몸','핫','거','넉','끼','단','접','호','창','난','칸','스','또한','듯','때문','린','오','시','은','함','적','샷',
    '이후','마치','가도','원','임','이제','얼마','밍','재','란','송','거나','구','중이','늘','감','앤','변','목','대해','근대','운','널리','곳도','가끔',
    '주','걸','장','상','이외','통해','얼','유','사','도량','갓','너','고기구','깃'
]

In [7]:
# 불용어 제거
for i in range(len(df)) :
    temp = [word for word in df.정보[i].split(' ') if word not in stop_word]
    df.loc[i, '정보'] = ' '.join(temp)

## TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['정보'])

In [9]:
import pickle

# 모델 저장
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

----

## 출력

In [10]:
# 사용자의 입력문장을 통한 코사인 유사도 계산
def user_input_to_cosine_sim(model, query) :
    query = ' '.join(ko.nouns(query))
    query_tfidf = model.transform([query])
    cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix).reshape(-1)
    return cosine_sim

# 장소명을 입력받아 그 장소명의 정보 추출
def searchByName(place_name) :
    # 찾을 정보의 df
    find_df = pd.read_csv("./data/구석구석_서울_크롤링_imageurl.csv")
    json_col = ["장소명", "주소", "상세설명", "문의 및 안내", "사진주소"]

    # temp_dict = find_df.loc[find_df.장소명 == place_name, json_col].fillna("정보 없음").iloc[0].to_dict()
    temp_df = find_df.loc[(find_df.장소명 == place_name), json_col].fillna("정보 없음")
    temp_df.columns = ["name", "address", "description", "inquieries", "imageName"]

    return temp_df.iloc[0].to_dict()

# 저장된 모델 불러오기
with open('./tfidf_vectorizer.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 사용자가 입력한 문장
input_sentence = "강남구 갈비집"


result_cosine_similarity = user_input_to_cosine_sim(loaded_model, input_sentence)
top_index = np.argsort(result_cosine_similarity)[::-1]

result_arr = []

for row in top_index[:10]:
    name = df.loc[row, '장소명']
    result_arr.append(searchByName(name))

In [10]:
# df의 모든 정보를 return
def find_all_info() :
    find_df = pd.read_csv("./data/구석구석_서울_크롤링_imageurl.csv")
    json_col = ["장소명", "주소", "상세설명", "문의 및 안내", "사진주소"]

    temp_df = find_df.loc[:, json_col].fillna("정보 없음")
    temp_df.columns = ["name", "address", "description", "inquiries", "imageName"]

    result_arr = []
    for i in range(len(temp_df)) :
        result_arr.append(temp_df.iloc[i,:].to_dict())

    return result_arr

----